In [1]:
import requests
import json
import os

from dotenv import load_dotenv

from datetime import datetime

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [2]:
load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

## Card Data
- Get card information via scryfall.com and delete unnecessary data: 
    - keep only: id, name, mana cost, converted mana cost, type, oracle text, colors, keywords
    - add comments via id
- save as .json

In [3]:
cards = ["Urborg, Tomb of Yawgmoth", "Blood Moon", "Humility", "Opalescence"]

for card in cards:
    response = requests.get(f'https://api.scryfall.com/cards/named?exact={card} ', 
                     headers={'Accept': 'application/json'})
    if response.status_code == 200:
        # Parse the JSON response into a Python dictionary to delete keywords
        data = response.json()
        keywords = ["object", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "rulings_uri", "set_search_uri", "scryfall_set_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
        for keyword in keywords:
            if keyword in data:
                del data[keyword]
            
        # convert into json dict
        card = json.loads(json.dumps(data))
        
        # check for additional rulings via id
        card_id = card.get('id')
        if card_id:
            response = requests.get(f'https://api.scryfall.com/cards/{card_id}/rulings', 
                     headers={'Accept': 'application/json'})
        
        # add additional rules to card
            if response.status_code == 200:
                data_rulings = response.json()
                comments = [item.get('comment') for item in data_rulings.get('data', [])]
                card['comments'] = comments
        
        card_name = card.get('name')
        with open(f'./data/cards/{card_name}.json', 'w') as f:
            json.dump(card, f)

## Prompt
- load prompt

In [41]:
with open("prompt_continuouseffects.txt", "r") as file:
    prompt_continuous_effects = file.read()
print(prompt_continuous_effects)

"""
You are judge (aka a referee) in a game of Magic: the Gathering. You will get a {question} regarding {card1} and {card2} and give an answer.
{card1} and {card2} each contain information of a single Magic: The Gathering card. Understanding the card helps you make a ruling decision.
Use the {rules_db} as context for your answer.

The following things are true:
- {card1} and {card2} both have continuous effects.
- Consider using the "comments" from {card1} and {card2}.
- The "type_line" of the card could be relevant for ruling decision.
- {card1} is played first and has the earlier timestamp, {card2} is played second and has the later timestamp. This may be relevant for the decision, but shouldn't influence your decision which rule to apply.


You will follow these steps to make a ruling decision:

1. Give the oracle texts from both cards as well the comments.
2. Determine the layer on which the effects of the cards are applied. Layers are defined in 613.1 in the {rules_db}.
3. Decide

## RAG
= Retrieved Augmented Generation
- ChatGPT's knowlegde of the rules is from January 2022: for current state the rulebook needs to be retrieved and passed to the LLM
- this data can provide context for the specific questions
### Loader
- simple loader to load .txt data: complete rule book from Magic: The Gathering

In [4]:
#loader = TextLoader("./data/rules_shortened.txt")
loader = TextLoader("./data/rules.txt")
rules_doc = loader.load()

### Splitter
- splitting the text into smaller chunks
- this text splitter splits text recursively by characters: ["\n\n", "\n", " ", ""], so first paragraphs, then sentences, then words
- keeps semantically related pieces together as long as possible

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
rules_splits = text_splitter.split_documents(rules_doc)
# print(rules_splits[0])

### Vector Store
- for storage of the embeddings
- embedding for capture semantic meaning of the text: that's how context is given for later question
- OpenAI embedding for ChatGPT
- Chroma as open-source embedding database
- finally, construct a retriever out of vectorstore 

In [7]:
rules_db = Chroma.from_documents(documents=rules_splits, embedding=OpenAIEmbeddings())
retriever_rules = rules_db.as_retriever()

- create new LLM: temperature 0 for more consistent, non-creative and therefor comparable output

In [8]:
model = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)

## Case 1: Blood Moon vs. Urborg, Tomb of Yawgmoth
- load card variables from saved data

In [38]:
with open("./data/cards/Urborg, Tomb of Yawgmoth.json", "r") as file:
    card1 = file.read()

with open("./data/cards/Blood Moon.json", "r") as file:
    card2 = file.read()

- load template with prompt and card variables

In [42]:
prompt_template = ChatPromptTemplate.from_template(
    template=prompt_continuous_effects,
    partial_variables={"card1": card1, "card2": card2},
)

- Outputparser to parse message response from Chatmodel to str
- check testcases.md to compare expected outcome with ChatGPT's answer

In [43]:
chain = ({"rules_db": retriever_rules, "question": RunnablePassthrough()} | prompt_template | model | StrOutputParser())
answer = chain.invoke("What is the final board state of two cards played with continuous effects from Magic: the Gathering?")
print(answer)

1. Oracle texts and comments:
- Urborg, Tomb of Yawgmoth:
    - Oracle text: Each land is a Swamp in addition to its other land types.
    - Comments: Urborg's ability causes each land on the battlefield to have the land type Swamp. Any land that's a Swamp has the ability "{T}: Add {B}". Nothing else changes about those lands, including their names, other subtypes, other abilities, and whether they're legendary, basic, or snow.
- Blood Moon:
    - Oracle text: Nonbasic lands are Mountains.
    - Comments: Nonbasic lands will lose any other land types and abilities they had. They will gain the land type Mountain and gain the ability "{T}: Add {R}". This effect doesn't affect names or supertypes.

2. Layer determination:
The effects of Urborg, Tomb of Yawgmoth and Blood Moon fall under the interaction of continuous effects, which are applied in layers as defined in 613.1.

3. Chosen rule: Layers (613.1 to 613.7)

4. Rule summary:
In Magic: The Gathering, continuous effects are applied in

- saving the answers as .txt for comparison by date & time: 

In [68]:
file_name = str(datetime.now().strftime("%Y-%m-%d %H-%M-%S"))

with open(f'./data/answers/case1_{file_name}.txt', 'w') as f:
    f.write(answer)

- cleanup

In [36]:
rules_db.delete_collection()

## Case 2a: Opalescence vs. Humility
- timestamp rule determines the outcome: *Humility* is played last and it's effect overrides *Opalescence*

In [16]:
with open("./data/cards/Opalescence.json", "r") as file:
    card1 = file.read()

with open("./data/cards/Humility.json", "r") as file:
    card2 = file.read()

In [17]:
prompt_template = ChatPromptTemplate.from_template(
    template=prompt_continuous_effects,
    partial_variables={"card1": card1, "card2": card2},
)

In [18]:
chain = ({"rules_db": retriever_rules, "question": RunnablePassthrough()} | prompt_template | model | StrOutputParser())
answer = chain.invoke("What is the final board state of two cards played with continuous effects from Magic: the Gathering?")
print(answer)

1. Oracle text and comments:
- Opalescence: Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value. This card does not animate itself but can be animated by another Opalescence.
- Humility: All creatures lose all abilities and have base power and toughness 1/1. This card removes all creature abilities, including mana abilities. Animated lands will also lose the ability to tap for mana.

2. Layers:
The effects of Opalescence and Humility are applied in different layers. Opalescence's effect of turning enchantments into creatures with power and toughness equal to their mana value is applied in layer 4, while Humility's effect of setting all creatures' power and toughness to 1/1 and removing all abilities is applied in layer 6.

3. Rule:
We will apply the rule of Layers (613.1 to 613.7) to determine the outcome of the continuous effects of Opalescence and Humility.

4. Summary of the rule:
In Magic: T

In [72]:
file_name = str(datetime.now().strftime("%Y-%m-%d %H-%M-%S"))

with open(f'./data/answers/case2_{file_name}.txt', 'w') as f:
    f.write(answer)

## Case 2b: Humility vs. Opalescence 
- because timestamp is important here, testing the case with changed order

In [19]:
with open("./data/cards/Humility.json", "r") as file:
    card1 = file.read()

with open("./data/cards/Opalescence.json", "r") as file:
    card2 = file.read()

In [20]:
prompt_template = ChatPromptTemplate.from_template(
    template=prompt_continuous_effects,
    partial_variables={"card1": card1, "card2": card2},
)

In [21]:
chain = ({"rules_db": retriever_rules, "question": RunnablePassthrough()} | prompt_template | model | StrOutputParser())
answer = chain.invoke("What is the outcome of two played cards with continuous effects from Magic: the Gathering?")
print(answer)

1. Oracle text and comments:
- Humility: All creatures lose all abilities and have base power and toughness 1/1. Comments include detailed interactions with Opalescence and the order of applying effects.
- Opalescence: Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value. Comments include interactions with Humility and the ability to animate other noncreature permanents.

2. Layers:
The effects of Humility and Opalescence are applied in different layers. Humility's effect of setting all creatures' power and toughness to 1/1 and removing all abilities is applied in layer 6, while Opalescence's effect of turning non-Aura enchantments into creatures with power and toughness equal to their mana value is applied in layer 4.

3. Rule choice:
The rule of overriding effects (613.9) will be applied in this scenario.

4. Rule summary:
In Magic: The Gathering, when two continuous effects are in conflict, th

In [76]:
file_name = str(datetime.now().strftime("%Y-%m-%d %H-%M-%S"))

with open(f'./data/answers/case2_{file_name}.txt', 'w') as f:
    f.write(answer)

- cleanup

In [19]:
rules_db.delete_collection()